# DataTalks MLOps Week 2 Homework


## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [1]:
import mlflow
!mlflow --version

mlflow, version 2.13.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7


In [2]:
# !wget -nv https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -O ./data/green/green_tripdata_2023-01.parquet
# !wget -nv https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet -O ./data/green/green_tripdata_2023-02.parquet
# !wget -nv https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet -O ./data/green/green_tripdata_2023-03.parquet

In [3]:
!python preprocess_data.py --raw_data_path ./data/green --dest_path ./output

In [4]:
!dir output

 Volume in drive C is Windows
 Volume Serial Number is CE80-71C3

 Directory of c:\Users\claim\Desktop\StackUp\ml-zoomcamp\week_2\output

05/29/2024  07:05 AM    <DIR>          .
05/29/2024  08:03 AM    <DIR>          ..
06/02/2024  01:16 PM           131,004 dv.pkl
06/02/2024  01:16 PM         2,458,697 test.pkl
06/02/2024  01:16 PM         2,374,517 train.pkl
06/02/2024  01:16 PM         2,215,823 val.pkl
               4 File(s)      7,180,041 bytes
               2 Dir(s)  261,047,197,696 bytes free


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10

In [29]:
import mlflow

client = mlflow.tracking.MlflowClient(tracking_uri = 'sqlite:///mlflow.db')
experiment = client.get_experiment_by_name('nyc-taxi-experiment')
runs = client.search_runs(1, order_by=["metrics.rmse ASC"])
print(f"{runs[0].data.params['min_samples_split']=}")

runs[0].data.params['min_samples_split']='2'


After running `python train.py` and `mlflow ui --backend-store-uri sqlite:///mlflow.db`, we see the value for `min_samples_split=2`.

## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`


When we run `mlflow server --backend-store-uri sqlite:///mlflow.db`, we get an error for not specifying artifacts folder.
The correct command is `mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts`.

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336

In [33]:
import mlflow

client = mlflow.tracking.MlflowClient(tracking_uri = 'sqlite:///mlflow.db')
experiment = client.get_experiment_by_name('random-forest-hyperopt')
runs = client.search_runs(experiment.experiment_id, order_by=['metrics.rmse ASC'])
print(f"{runs[0].data.metrics['rmse']}")

5.335419588556921


We observe that the rmse 5.335 is the solution.

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [58]:
import mlflow

client = mlflow.tracking.MlflowClient(tracking_uri = 'sqlite:///mlflow.db')
mlflow.set_tracking_uri("http://127.0.0.1:5000")
model_name = "Random_Forest_Regressor_hw2"
latest_versions = client.get_latest_versions(name=model_name)
# print(latest_versions)
client.download_artifacts(run_id=latest_versions[0].run_id, path='.', dst_path='.')


C:\Users\claim\AppData\Local\Temp\ipykernel_26548\519455913.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1717523157749, current_stage='None', description='', last_updated_timestamp=1717523157749, name='Random_Forest_Regressor_hw2', run_id='22c7e3f315a846e38fdedb0736729513', run_link='', source='models:/Random_Forest_Regressor_hw2/2', status='READY', status_message=None, tags={}, user_id=None, version=3>]


'c:\\Users\\claim\\Desktop\\StackUp\\ml-zoomcamp\\week_2\\.'

In [62]:
import pickle
import os
from sklearn.metrics import root_mean_squared_error

def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)
source = latest_versions[0].source
X_test, y_test = load_pickle(os.path.join("./output", "test.pkl"))
model = mlflow.pyfunc.load_model(source)
print(model)
y_pred = model.predict(X_test)
print({"rmse": root_mean_squared_error(y_test, y_pred)})

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 22c7e3f315a846e38fdedb0736729513

{'rmse': 5.567408012462019}


We observe that the rmse is 5.567, which is one of the options.